# 手書き文字の認識
## mnist (Keras + PlaidML版)

In [27]:
# PlaidML の利用
import plaidml.keras
plaidml.keras.install_backend()

In [28]:
# Keras の読み込み
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# 学習パラメーターの設定
batch_size = 128
num_classes = 10
epochs = 12

# 画像サイズ
img_rows, img_cols = 28, 28

In [30]:
# MNIST データセットの読み込み
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# データの変換
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [31]:
# モデルの作成
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [32]:
# MNIST の画像で CNN　を訓練
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 63s 1ms/step - loss: 0.2725 - acc: 0.9155 - val_loss: 0.0567 - val_acc: 0.9816
Epoch 2/12
60000/60000 [==============================] - 46s 770us/step - loss: 0.0895 - acc: 0.9731 - val_loss: 0.0403 - val_acc: 0.9872
Epoch 3/12
60000/60000 [==============================] - 48s 795us/step - loss: 0.0652 - acc: 0.9807 - val_loss: 0.0358 - val_acc: 0.9877
Epoch 4/12
60000/60000 [==============================] - 46s 771us/step - loss: 0.0537 - acc: 0.9844 - val_loss: 0.0357 - val_acc: 0.9881
Epoch 5/12
60000/60000 [==============================] - 46s 770us/step - loss: 0.0469 - acc: 0.9858 - val_loss: 0.0304 - val_acc: 0.9907
Epoch 6/12
60000/60000 [==============================] - 46s 773us/step - loss: 0.0390 - acc: 0.9879 - val_loss: 0.0286 - val_acc: 0.9902
Epoch 7/12
60000/60000 [==============================] - 46s 773us/step - loss: 0.0363 - acc: 0.9889 - val_loss: 0.02

In [33]:
# 手書き数字の識別
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
from IPython.core.display import Image, display

# 予測関数の定義
def predict(f):
    # 画像の読み込み
    x = img_to_array(load_img(f, target_size=(28, 28), color_mode="grayscale"))
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    # 予測
    pred = model.predict(x)[0]

    # 予測結果の表示
    for i in range(len(pred)):
        print(i, ":", pred[i])

# ファイルの指定
img_files = ['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']

# 各ファイルについて予測
for f in img_files:
    # 画像の表示
    print("----------")
    print(f)
    display(Image(f, width=150, unconfined=True))

    # 予測の実行
    predict(f)

----------
0.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
1.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
2.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
3.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
4.jpg


...

0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
6.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
7.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
8.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
----------
9.jpg


0 : 0.09842857
1 : 2.1131147e-05
2 : 0.29696506
3 : 0.46256015
4 : 1.772207e-08
5 : 0.00028912266
6 : 2.8707616e-06
7 : 0.00015848312
8 : 2.5859968e-05
9 : 0.14154863
